## Notebook to bulk download Planet imagery through the API
Rainey Aberle, Fall 2021

Modified from [Planet Developers API Tutorial](https://developers.planet.com/docs/apis/data/) and [Planet Labs: `ordering_and_delivery.ipynb`](https://github.com/planetlabs/notebooks/blob/master/jupyter-notebooks/orders/ordering_and_delivery.ipynb )


__To-Do:__
- View image thumbnails before downloading. Will need to download only the metadata first, then plot the coordinates over a regional map. 
- Implement percent AOI coverage filter for image search

### Import necessary packages

In [16]:
import os
import glob
import json
import requests
import time
import geopandas as gpd
from shapely import geometry as sgeom
from pathlib import Path
import rasterio as rio
import numpy as np
import sys
from rasterio.plot import show
from requests.auth import HTTPBasicAuth
# add path to functions
sys.path.insert(1, '/Users/maddiegendreau/Documents/GitHub/planet_tile2img/')
import PlanetScope_orders_utils as orders

### Install Planet API Client

This will allow you to interact with the Planet API through this notebook. Refer to the __[Planet API documentation](https://developers.planet.com/docs/apis/data/)__ for more info. 

In [13]:
# !conda install -c conda-forge rasterio -y

### Define filters for image search
#### _Modify these sections_

In [27]:
#### OPTION 2: Import an existing shapefile

# Name of your file
# If your shapefile is not currently in this directory, you need to include the full file path in 'file_name' below
file_name = '/Volumes/SGlacier/BoxTurner/BoxTurner.shp'

# Read in the shapefile
AOI = gpd.read_file(file_name)

# File extension index (we don't want the .shp extension in the next line)
i = file_name.index('.shp')

# Convert to geojson
AOI.to_file(file_name[0:i]+'.geojson', driver='GeoJSON')

# Adjust AOI polygon to a rectangular shape 
# Planet only excepts a bounding BOX as a spatial filter, 
# so we need to convert our AOI to a box (if it is not already). 
AOI_box = {"type": "Polygon",
           "coordinates": [[
               [AOI.bounds.minx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.miny[0]],
               [AOI.bounds.maxx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.maxy[0]],
               [AOI.bounds.minx[0],AOI.bounds.miny[0]]
           ]]
          }
AOI_box_shape = sgeom.shape(AOI_box)
AOI_box

/opt/miniconda3/envs/autorift/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


{'type': 'Polygon',
 'coordinates': [[[-140.1698068780743, 59.904177627649],
   [-139.42646300960544, 59.904177627649],
   [-139.42646300960544, 60.27836331801325],
   [-140.1698068780743, 60.27836331801325],
   [-140.1698068780743, 59.904177627649]]]}

In [28]:
# ----------AOI clipping----------
# Would you like to clip images to the AOI (True/False)?
clip_AOI = True

# ----------Date Range----------
# Format: 'YYYY-MM-DD'
start_date = "2020-03-01"
end_date = "2020-03-31"

# ----------Cloud Filter----------
# Format: decimal (e.g., 50% max cloud cover = 0.5)
max_cloud_cover = 0.8

# ----------Item Type----------
# See here for possible image ("item") types:
# https://developers.planet.com/docs/apis/data/items-assets/
# item_type = "PSScene4Band" #OLD (now deprecated)
item_type = "PSScene"
asset_type = "ortho_analytic_4b_sr"

# ----------Planet API Key----------
# Find your API key on your Planet Account > My Settings > API Key
API_key = 'c2e92a042f6744eba732c282d09539f8'

# ----------Output folder----------
# AKA, where you want your images to be downloaded in your directory
out_folder = '/Volumes/SGlacier/TG_2020/'

### Authentication via basic HTTP

In [29]:
# set API key as environment variable
os.environ['PL_API_KEY'] = API_key

# Setup the API Key stored as the `PL_API_KEY` environment variable
PLANET_API_KEY = os.getenv('PL_API_KEY')

# Orders URL
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

# Authorize
auth = HTTPBasicAuth(PLANET_API_KEY, '')
response = requests.get(orders_url, auth=auth)
response

<Response [200]>

### Compile filters and use Quick Search to grab image IDs

In [30]:
# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": AOI_box
}

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": start_date + "T00:00:00.000Z",
    "lte": end_date + "T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": max_cloud_cover
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

# define the clip tool
clip = {
    "clip": {
        "aoi": AOI_box
    }
}

# -----AOI clipping
# Determine whether to clip images to the AOI (True/False)
# This greatly speeds up the ordering and downloading process and decreases the usage of your imagery quota
clip_to_AOI = True

# -----Sentinel-2 Harmonization
# option to harmonize PlanetScope imagery to Sentinel-2
harmonize = True # = True to harmonize

# -----Create request
QS_request = orders.build_QS_request(AOI_box_shape, max_cloud_cover, start_date, end_date, item_type, asset_type)
        
# -----Planet API Quick Search using created request
# fire off the POST request
QS_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(PLANET_API_KEY, ''),
    json=QS_request)
# Print resulting image IDs
im_ids = [feature['id'] for feature in QS_result.json()['features']]
im_ids = sorted(im_ids)
print(len(im_ids),'images found')

201 images found


In [31]:
# only download images that don't already exist in directory
im_ids_filtered = []
for im_id in im_ids:
    num_exist = len(glob.glob(out_folder + im_id+'_SR_clip.tif'))
    if num_exist==0:
        im_ids_filtered = im_ids_filtered + [im_id]
print(str(len(im_ids_filtered)) + ' new images to be downloaded')

201 new images to be downloaded


## (NEW) Place Order based on Rainey's code

In [32]:
# -----Build new request
request = orders.build_request_itemIDs(AOI_box, clip_to_AOI, harmonize, im_ids_filtered, item_type, asset_type)

# -----Place order
if orders_url!="N/A":
    order_url = orders.place_order(orders_url, request, auth)

{'_links': {'_self': 'https://api.planet.com/compute/ops/orders/v2/fbb42cf7-3642-4343-a924-d0c105bbbac0'}, 'created_on': '2023-04-06T22:57:48.416Z', 'error_hints': [], 'id': 'fbb42cf7-3642-4343-a924-d0c105bbbac0', 'last_message': 'Preparing order', 'last_modified': '2023-04-06T22:57:48.416Z', 'name': 'simple order', 'products': [{'item_ids': ['20200301_201048_0f4e', '20200303_200904_1014', '20200303_204237_10_1058', '20200305_192536_43_106f', '20200305_201416_1012', '20200305_201417_1012', '20200305_201419_1012', '20200305_201420_1012', '20200305_201421_1012', '20200305_201422_1012', '20200305_201423_1012', '20200306_192606_44_106b', '20200306_192612_63_106b', '20200306_200740_0e3a', '20200306_200742_0e3a', '20200306_200743_0e3a', '20200306_200744_0e3a', '20200306_201501_0f25', '20200306_201502_0f25', '20200306_201503_0f25', '20200306_201505_0f25', '20200306_201506_0f25', '20200306_204119_78_105d', '20200306_204121_82_105d', '20200306_204123_85_105d', '20200309_201838_1011', '20200311_

### Poll for Order Success
- This section outputs the status of the order every ~10 sec. This will take a few minutes... 
- Wait until it outputs `success` to proceed to the next section. It will stop after 30 loops, so try proceeding to the next section if it finishes running and does not output `success`.
- If you are ordering a LOT of images, consider narrowing your date range to download less images at a time. 

In [33]:
# -----Poll the order every 10s until it outputs "success," "failed," or "partial"
# Only continue to the next cell if outputs "success". Otherwise, try again or submit a new search request. 
orders.poll_for_success(order_url, auth)

queued
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
running
r

In [34]:
# -----View results
r = requests.get(order_url, auth=auth)
response = r.json()
results = response['_links']['results']
# print all files to be downloaded from order
[r['name'] for r in results]

['fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200306_201505_0f25_metadata.json',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200306_201505_0f25_3B_AnalyticMS_metadata_clip.xml',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200306_201505_0f25_3B_AnalyticMS_SR_harmonized_clip.tif',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200306_201505_0f25_3B_udm2_clip.tif',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200305_201416_1012_3B_AnalyticMS_metadata_clip.xml',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200305_201416_1012_metadata.json',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200305_201416_1012_3B_udm2_clip.tif',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200305_201416_1012_3B_AnalyticMS_SR_harmonized_clip.tif',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200305_201420_1012_3B_AnalyticMS_metadata_clip.xml',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/20200305_201420_1012_3B_udm2_clip.tif',
 'fbb42cf7-3642-4343-a924-d0c105bbbac0/PSScene/2020

In [35]:
orders.download_results(results, out_folder)

805 items to download


100%|█████████████████████████████████████████| 805/805 [18:57<00:00,  1.41s/it]

Done!


## You did it!

<div>
<img src="sandy-cheeks.jpeg" width="400"/>
</div>